In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 4.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=c5a12f49656b74a9d692923f0894857f709679a8717c08fba0c19e0620d3b76b
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenh

In [ ]:
from pyspark.sql import SparkSession
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd
from pyspark.ml.fpm import FPGrowth

In [ ]:
sc = pyspark.SparkContext()
spark = SparkSession.builder.getOrCreate()

In [ ]:
# read the dataset into a DataFrame and split the items into an array column
rdd = spark.sparkContext.textFile("browsing.txt")
transactions = rdd.map(lambda line: line.strip().split(' ')).toDF("")
from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.append('/content/drive/My Drive')

Mounted at /content/gdrive


In [ ]:
# Load the browsing.txt file into a dataframe
data = spark.read.text("/content/gdrive/MyDrive/Colab Notebooks/CSC 4008/browsing.txt")

# Split each line of the dataframe into individual items
transactions = data.selectExpr("filter(split(value, ' '), x -> x != '') as items")
transactions = transactions.select(array_distinct("items").alias("items"))

In [ ]:
transactions.take(1)

[Row(items=['FRO11987', 'ELE17451', 'ELE89019', 'SNA90258', 'GRO99222'])]

In [ ]:
N = transactions.count()
s = 100
min_support = s/N

In [ ]:
fpGrowth = FPGrowth(itemsCol="items", minSupport=min_support, minConfidence=0.0)
model = fpGrowth.fit(transactions)

In [ ]:
freq_itemsets = model.freqItemsets.filter(size(col("items")) == 1)
freq_itemsets.count()

647

In [ ]:
rules_set = model.associationRules.filter(size(col("antecedent")) == 1)
rules_set.sort(["confidence","antecedent"], ascending=[False, True]).show()

+----------+----------+------------------+------------------+--------------------+
|antecedent|consequent|        confidence|              lift|             support|
+----------+----------+------------------+------------------+--------------------+
|[DAI93865]|[FRO40251]|               1.0| 8.013656274156146|0.006687887849265297|
|[GRO85051]|[FRO40251]| 0.999176276771005|  8.00705523933394|0.039001961351725026|
|[GRO38636]|[FRO40251]|0.9906542056074766| 7.938762290285528|0.003408250538567...|
|[ELE12951]|[FRO40251]|0.9905660377358491|  7.93805574326788| 0.00337609723160027|
|[DAI88079]|[FRO40251]|0.9867256637168141| 7.907280305915135|0.014340374907559243|
|[FRO92469]|[FRO40251]| 0.983510011778563| 7.881511176584666|0.026848011317964052|
|[DAI43868]|[SNA82528]| 0.972972972972973|50.943488943488944|0.009260152406675026|
|[DAI23334]|[DAI62779]|0.9545454545454546| 4.452875083518551|0.008777852802160702|
|[ELE92920]|[DAI62779]|0.7326649958228906|3.4178212142024478| 0.02819845021060416|
|[DA

In [ ]:
rules_set.sort(["confidence","antecedent"], ascending=[False, True]).limit(5).show()

+----------+----------+------------------+-----------------+--------------------+
|antecedent|consequent|        confidence|             lift|             support|
+----------+----------+------------------+-----------------+--------------------+
|[DAI93865]|[FRO40251]|               1.0|8.013656274156146|0.006687887849265297|
|[GRO85051]|[FRO40251]| 0.999176276771005| 8.00705523933394|0.039001961351725026|
|[GRO38636]|[FRO40251]|0.9906542056074766|7.938762290285528|0.003408250538567...|
|[ELE12951]|[FRO40251]|0.9905660377358491| 7.93805574326788| 0.00337609723160027|
|[DAI88079]|[FRO40251]|0.9867256637168141|7.907280305915135|0.014340374907559243|
+----------+----------+------------------+-----------------+--------------------+



In [ ]:
freq_itemsets = model.freqItemsets.filter(size(col("items")) == 3)
freq_itemsets.count()

233

In [ ]:
rules_set = model.associationRules.filter(size(col("antecedent")) == 2)
rules_set = rules_set.withColumn('antecedent', sort_array(rules_set.antecedent))
rules_set.sort(["confidence","antecedent"], ascending=[False, True]).show()

+--------------------+----------+------------------+-----------------+--------------------+
|          antecedent|consequent|        confidence|             lift|             support|
+--------------------+----------+------------------+-----------------+--------------------+
|[DAI23334, ELE92920]|[DAI62779]|               1.0|4.664916754162292|0.004597922896369...|
|[DAI31081, GRO85051]|[FRO40251]|               1.0|8.013656274156146|0.003279637310697405|
|[DAI55911, GRO85051]|[FRO40251]|               1.0|8.013656274156146|0.004276389826693...|
|[DAI62779, DAI88079]|[FRO40251]|               1.0|8.013656274156146|0.003761936915211...|
|[DAI75645, GRO85051]|[FRO40251]|               1.0|8.013656274156146| 0.01270055625221054|
|[ELE17451, GRO85051]|[FRO40251]|               1.0|8.013656274156146|0.006977267611973891|
|[ELE20847, FRO92469]|[FRO40251]|               1.0|8.013656274156146|0.003922703450049837|
|[ELE20847, GRO85051]|[FRO40251]|               1.0|8.013656274156146|0.00446930

In [ ]:
rules_set.sort(["confidence","antecedent"], ascending=[False, True]).limit(5).show()

+--------------------+----------+----------+-----------------+--------------------+
|          antecedent|consequent|confidence|             lift|             support|
+--------------------+----------+----------+-----------------+--------------------+
|[DAI23334, ELE92920]|[DAI62779]|       1.0|4.664916754162292|0.004597922896369...|
|[DAI31081, GRO85051]|[FRO40251]|       1.0|8.013656274156146|0.003279637310697405|
|[DAI55911, GRO85051]|[FRO40251]|       1.0|8.013656274156146|0.004276389826693...|
|[DAI62779, DAI88079]|[FRO40251]|       1.0|8.013656274156146|0.003761936915211...|
|[DAI75645, GRO85051]|[FRO40251]|       1.0|8.013656274156146| 0.01270055625221054|
+--------------------+----------+----------+-----------------+--------------------+

